In [2]:
from prepare_dataset import prepare_dataset
import torch.utils.data as data_utils

data = prepare_dataset()
train_set = data[0]
valid_set = data[1]

train_loader = data_utils.DataLoader(train_set, batch_size=64, pin_memory=True, shuffle=True)
valid_loader = data_utils.DataLoader(valid_set, batch_size=64,)

from torch.utils.data import ConcatDataset

dataloaders = {'train':train_loader,'valid':valid_loader}
dataset_sizes = {x: len(dataloaders[x]) for x in ['train', 'valid']}
dataset = ConcatDataset([train_set, valid_set])

[1 1 1 ... 1 1 1]
Number of positive samples in training data: 1605 (50.03% of total)
Number of positive samples in validation data: 395 (49.87% of total)


In [3]:
from sklearn.model_selection import KFold
from model import EfficientNet
import torch
from tqdm import tqdm
from torch.autograd import Variable
device = torch.device("mps")

kfold = KFold(n_splits=5, shuffle=True)
for fold, (train_idx, valid_idx) in enumerate(kfold.split(dataset)):

    model = EfficientNet.from_name('efficientnet-phospho-B-15')
    model = model.to(device)
   # Define data loaders for training and testing data in this fold
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    print(len(train_idx))
    print(len(valid_idx))
    test_subsampler = torch.utils.data.SubsetRandomSampler(valid_idx)
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                dataset,
                batch_size=64, sampler=train_subsampler)
    validloader = torch.utils.data.DataLoader(
                dataset,
                batch_size=64, sampler=test_subsampler)

    for epoch in tqdm(range(20), position=0, leave=True):
        # print('-' * 60)
        # print('Epoch {}/{}'.format(epoch+1, 20))

        train_corrects = 0.0
        train_loss = 0.0
        train_precision, train_recall, train_f1 = 0.0, 0.0, 0.0

        for _, (inputs, labels) in enumerate(tqdm(trainloader, position=1, leave=True)):
                model.train(True)
                inputs = Variable(inputs.to(device, dtype=torch.float), requires_grad=True)
                labels = Variable(labels.to(device))
                pred = model(inputs) # forward

efficientnet-phospho-B-15
3200
800


  0%|          | 0/20 [00:00<?, ?it/s]


RuntimeError: Given groups=32, weight of size [32, 1, 3, 3], expected input[64, 8, 134, 17] to have 32 channels, but got 8 channels instead

In [1]:
import re
import collections
import torch
from torch import nn
from utils import *
from torch.nn import functional as F

GlobalParams = collections.namedtuple(
    "GlobalParams",
    [
        "width_coefficient",
        "depth_coefficient",
        "image_size",
        "dropout_rate",
        "num_classes",
        "batch_norm_momentum",
        "batch_norm_epsilon",
        "drop_connect_rate",
        "depth_divisor",
        "min_depth",
        "include_top",
    ],
)

# Parameters for an individual model block
BlockArgs = collections.namedtuple(
    "BlockArgs",
    [
        "num_repeat",
        "kernel_size",
        "stride",
        "expand_ratio",
        "input_filters",
        "output_filters",
        "se_ratio",
        "id_skip",
    ],
)

# Set GlobalParams and BlockArgs's defaults
GlobalParams.__new__.__defaults__ = (None,) * len(GlobalParams._fields)
BlockArgs.__new__.__defaults__ = (None,) * len(BlockArgs._fields)

In [226]:
def efficientnet(width_coefficient=None, depth_coefficient=None, dropout_rate=0.2,
                 drop_connect_rate=0.2, image_size=None, num_classes=1):
    """ Creates a efficientnet model. """

    blocks_args = [
        'r1_kh8_kw2_sh2_sw2_e2_i8_o16_se0.25',
        'r1_kh8_kw3_sh2_sw1_e2_i16_o32_se0.25',
        'r1_kh6_kw3_sh2_sw2_e2_i32_o64_se0.25',
        # 'r3_k3_s22_e6_i40_o80_se0.25',
        # 'r3_k5_s11_e6_i80_o112_se0.25',
        # 'r4_k5_s22_e6_i112_o192_se0.25',
        # 'r1_k3_s11_e6_i192_o320_se0.25',
        # 'r1_k3_s11_e6_i24_o48_se0.25',
    ]
    blocks_args = BlockDecoder.decode(blocks_args)

    global_params = GlobalParams(
        batch_norm_momentum=0.99,
        batch_norm_epsilon=1e-3,
        dropout_rate=dropout_rate,
        drop_connect_rate=drop_connect_rate,
        # data_format='channels_last',  # removed, this is always true in PyTorch
        num_classes=num_classes,
        width_coefficient=width_coefficient,
        depth_coefficient=depth_coefficient,
        depth_divisor=8,
        min_depth=None,
        image_size=image_size,
    )

    return blocks_args, global_params

blocks_args, global_params = efficientnet(width_coefficient=1.0, depth_coefficient=1.0, dropout_rate=0.7, image_size=[263, 15])

In [227]:
print(global_params)
print()
for idx, block_args in enumerate(blocks_args):
    print(f'MBConvblock {idx}: ', block_args)
    print()

GlobalParams(width_coefficient=1.0, depth_coefficient=1.0, image_size=[263, 15], dropout_rate=0.7, num_classes=1, batch_norm_momentum=0.99, batch_norm_epsilon=0.001, drop_connect_rate=0.2, depth_divisor=8, min_depth=None, include_top=None)

MBConvblock 0:  BlockArgs(num_repeat=1, kernel_size=(8, 2), stride=(2, 2), expand_ratio=2, input_filters=8, output_filters=16, se_ratio=0.25, id_skip=True)

MBConvblock 1:  BlockArgs(num_repeat=1, kernel_size=(8, 3), stride=(2, 1), expand_ratio=2, input_filters=16, output_filters=32, se_ratio=0.25, id_skip=True)

MBConvblock 2:  BlockArgs(num_repeat=1, kernel_size=(6, 3), stride=(2, 2), expand_ratio=2, input_filters=32, output_filters=64, se_ratio=0.25, id_skip=True)



In [228]:
def calculate_output_image_size_(input_image_size, stride):
    """Calculates the output image size when using Conv2dSamePadding with a stride.
       Necessary for static padding. Thanks to mannatsingh for pointing this out.

    Args:
        input_image_size (int, tuple or list): Size of input image.
        stride (int, tuple or list): Conv2d operation's stride.

    Returns:
        output_image_size: A list [H,W].
    """
    if input_image_size is None:
        return None
    image_height, image_width = get_width_and_height_from_size(input_image_size)
    sh, sw = stride if type(stride) == (list) or (tuple) else [stride, stride]
    # stride = stride if isinstance(stride, int) else stride[0]
    image_height = int(math.ceil(image_height / sh))
    image_width = int(math.ceil(image_width / sw))
    return [image_height, image_width]

In [236]:
bn_mom = 1 - global_params.batch_norm_momentum
bn_eps = global_params.batch_norm_epsilon
image_size = global_params.image_size
Conv2d = partial(Conv2dStaticSamePadding, image_size=image_size)

In [237]:
# Stem
image_size = [263, 15]
in_channels = 1
out_channels = round_filters(8, global_params)
_conv_stem = Conv2d(in_channels, out_channels, kernel_size=(15, 1), stride=(2, 1), bias=False, image_size=image_size)
_bn0 = nn.BatchNorm2d(num_features=out_channels, momentum=bn_mom, eps=bn_eps)
image_size = calculate_output_image_size_(image_size, stride=(2, 1))
image_size

[132, 15]

In [238]:
_swish = MemoryEfficientSwish()
x0 = torch.randn((1, 1, 263, 15))
x1 = _conv_stem(x0)
x2 = _bn0(x1)
x3 = _swish(x2)

In [240]:
x3.shape

torch.Size([1, 8, 132, 15])

In [232]:
for idx, block_args in enumerate(blocks_args):
    print(f'MBConvblock {idx}: ', block_args)
    print()

MBConvblock 0:  BlockArgs(num_repeat=1, kernel_size=(8, 2), stride=(2, 2), expand_ratio=2, input_filters=8, output_filters=16, se_ratio=0.25, id_skip=True)

MBConvblock 1:  BlockArgs(num_repeat=1, kernel_size=(8, 3), stride=(2, 1), expand_ratio=2, input_filters=16, output_filters=32, se_ratio=0.25, id_skip=True)

MBConvblock 2:  BlockArgs(num_repeat=1, kernel_size=(6, 3), stride=(2, 2), expand_ratio=2, input_filters=32, output_filters=64, se_ratio=0.25, id_skip=True)



In [191]:
# 263, 15 -> 132, 8 -> 64, 15 -> 32, 15 -> 17, 4

In [233]:
class MBConvBlock(nn.Module):
    """
    Mobile Inverted Residual Bottleneck Block
    Args:
        block_args (namedtuple): BlockArgs, see above
        global_params (namedtuple): GlobalParam, see above
    Attributes:
        has_se (bool): Whether the block contains a Squeeze and Excitation layer.
    """

    def __init__(self, block_args, global_params, image_size=None):
        super().__init__()
        self._block_args = block_args
        self._bn_mom = 1 - global_params.batch_norm_momentum
        self._bn_eps = global_params.batch_norm_epsilon
        self.has_se = (self._block_args.se_ratio is not None) and (
            0 < self._block_args.se_ratio <= 1
        )
        self.id_skip = block_args.id_skip  # skip connection and drop connect

        # Expansion phase (Inverted Bottleneck)
        inp = self._block_args.input_filters  # number of input channels
        oup = (
            self._block_args.input_filters * self._block_args.expand_ratio
        )  # number of output channels
        if self._block_args.expand_ratio != 1:
            Conv2d = partial(Conv2dStaticSamePadding, image_size=image_size)
            self._expand_conv = Conv2d(
                in_channels=inp, out_channels=oup, kernel_size=1, bias=False
            )
            self._bn0 = nn.BatchNorm2d(
                num_features=oup, momentum=self._bn_mom, eps=self._bn_eps
            )
            # image_size = calculate_output_image_size(image_size, 1) <-- this wouldn't modify image_size

        # Depthwise convolution phase
        k = self._block_args.kernel_size
        s = self._block_args.stride
        Conv2d = partial(Conv2dStaticSamePadding, image_size=image_size)
        self._depthwise_conv = Conv2d(
            in_channels=oup,
            out_channels=oup,
            groups=oup,  # groups makes it depthwise
            kernel_size=k,
            stride=s,
            bias=False,
        )
        self._bn1 = nn.BatchNorm2d(
            num_features=oup, momentum=self._bn_mom, eps=self._bn_eps
        )
        image_size = calculate_output_image_size_(image_size, s)

        # Squeeze and Excitation layer, if desired
        if self.has_se:
            Conv2d = get_same_padding_conv2d(image_size=(1, 1))
            num_squeezed_channels = max(
                1, int(self._block_args.input_filters * self._block_args.se_ratio)
            )
            self._se_reduce = Conv2d(
                in_channels=oup, out_channels=num_squeezed_channels, kernel_size=1
            )
            self._se_expand = Conv2d(
                in_channels=num_squeezed_channels, out_channels=oup, kernel_size=1
            )

        # Output phase (Pointwise convolution phase)
        final_oup = self._block_args.output_filters
        Conv2d = partial(Conv2dStaticSamePadding, image_size=image_size)
        self._project_conv = Conv2d(
            in_channels=oup, out_channels=final_oup, kernel_size=1, bias=False
        )
        self._bn2 = nn.BatchNorm2d(
            num_features=final_oup, momentum=self._bn_mom, eps=self._bn_eps
        )
        self._swish = MemoryEfficientSwish()

    def forward(self, inputs, drop_connect_rate=None):
        """
        :param inputs: input tensor
        :param drop_connect_rate: drop connect rate (float, between 0 and 1)
        :return: output of block
        """

        # Expansion and Depthwise Convolution
        x = inputs
        if self._block_args.expand_ratio != 1:
            x = self._expand_conv(inputs)
            x = self._bn0(x)
            x = self._swish(x)

        x = self._depthwise_conv(x)
        x = self._bn1(x)
        x = self._swish(x)

        # Squeeze and Excitation
        if self.has_se:
            x_squeezed = F.adaptive_avg_pool2d(x, 1)
            x_squeezed = self._se_reduce(x_squeezed)
            x_squeezed = self._swish(x_squeezed)
            x_squeezed = self._se_expand(x_squeezed)
            x = torch.sigmoid(x_squeezed) * x

        # Pointwise Convolution
        x = self._project_conv(x)
        x = self._bn2(x)

        # Skip connection and drop connect
        input_filters, output_filters = (
            self._block_args.input_filters,
            self._block_args.output_filters,
        )
        if (
            self.id_skip
            and self._block_args.stride == 1
            and input_filters == output_filters
        ):
            # The combination of skip connection and drop connect brings about stochastic depth.
            if drop_connect_rate:
                x = drop_connect(x, p=drop_connect_rate, training=self.training)
            x = x + inputs  # skip connection
        return x

    def set_swish(self, memory_efficient=True):
        """Sets swish function as memory efficient (for training) or standard (for export).

        Args:
            memory_efficient (bool): Whether to use memory-efficient version of swish.
        """
        self._swish = MemoryEfficientSwish() if memory_efficient else Swish()

In [194]:
_bn_mom = 1 - global_params.batch_norm_momentum
_bn_eps = global_params.batch_norm_epsilon

has_se = (block_args.se_ratio is not None) and (0 < block_args.se_ratio <= 1)
id_skip = block_args.id_skip

In [195]:
# Expansion phase (Inverted Bottleneck)
inp = block_args.input_filters  # number of input channels
oup = (
    block_args.input_filters * block_args.expand_ratio
)  # number of output channels
if block_args.expand_ratio != 1:
    Conv2d = partial(Conv2dStaticSamePadding, image_size=image_size)
    _expand_conv = Conv2d(
        in_channels=inp, out_channels=oup, kernel_size=1, bias=False
    )
    _bn0 = nn.BatchNorm2d(
        num_features=oup, momentum=_bn_mom, eps=_bn_eps
    )

In [196]:
# Depthwise convolution phase
k = block_args.kernel_size
s = block_args.stride
Conv2d = partial(Conv2dStaticSamePadding, image_size=image_size)
_depthwise_conv = Conv2d(
    in_channels=oup,
    out_channels=oup,
    groups=oup,  # groups makes it depthwise
    kernel_size=k,
    stride=s,
    bias=False,
)
_bn1 = nn.BatchNorm2d(
    num_features=oup, momentum=_bn_mom, eps=_bn_eps
)
image_size = calculate_output_image_size_(image_size, s)

In [197]:
# Squeeze and Excitation layer, if desired
if has_se:
    Conv2d = get_same_padding_conv2d(image_size=(1, 1))
    num_squeezed_channels = max(
        1, int(block_args.input_filters * block_args.se_ratio)
    )
    _se_reduce = Conv2d(
        in_channels=oup, out_channels=num_squeezed_channels, kernel_size=1
    )
    _se_expand = Conv2d(
        in_channels=num_squeezed_channels, out_channels=oup, kernel_size=1
    )

# Output phase (Pointwise convolution phase)
final_oup = block_args.output_filters
Conv2d = partial(Conv2dStaticSamePadding, image_size=image_size)
_project_conv = Conv2d(
    in_channels=oup, out_channels=final_oup, kernel_size=1, bias=False
)
_bn2 = nn.BatchNorm2d(
    num_features=final_oup, momentum=_bn_mom, eps=_bn_eps
)
_swish = MemoryEfficientSwish()

In [198]:
block_args

BlockArgs(num_repeat=1, kernel_size=(8, 2), stride=(2, 2), expand_ratio=1, input_filters=8, output_filters=16, se_ratio=0.25, id_skip=True)

In [215]:
inputs = x3
inputs.shape
x = inputs

In [216]:
x = _depthwise_conv(x)
print(x.shape)
x = _bn1(x)
print(x.shape)
x = _swish(x)
print(x.shape)

torch.Size([1, 8, 66, 7])
torch.Size([1, 8, 66, 7])
torch.Size([1, 8, 66, 7])


In [217]:
if has_se:
    x_squeezed = F.adaptive_avg_pool2d(x, 1)
    print(x_squeezed.shape)
    x_squeezed = _se_reduce(x_squeezed)
    print(x_squeezed.shape)
    x_squeezed = _swish(x_squeezed)
    print(x_squeezed.shape)
    x_squeezed = _se_expand(x_squeezed)
    print(x_squeezed.shape)
    x = torch.sigmoid(x_squeezed) * x
    print(x.shape)

torch.Size([1, 8, 1, 1])
torch.Size([1, 2, 1, 1])
torch.Size([1, 2, 1, 1])
torch.Size([1, 8, 1, 1])
torch.Size([1, 8, 66, 7])


In [224]:
x.shape

torch.Size([1, 16, 66, 7])

In [218]:
x = _project_conv(x)

In [219]:
print(x.shape)

torch.Size([1, 16, 66, 7])


In [220]:
# Pointwise Convolution
# x = _project_conv

x = _bn2(x)
print(x.shape)

torch.Size([1, 16, 66, 7])


In [221]:
# Skip connection and drop connect
input_filters, output_filters = (
    block_args.input_filters,
    block_args.output_filters,
)
input_filters, output_filters

(8, 16)

In [ ]:

if (
    id_skip
    and block_args.stride == 1
    and input_filters == output_filters
):
    # The combination of skip connection and drop connect brings about stochastic depth.
    if drop_connect_rate:
        x = drop_connect(x, p=drop_connect_rate, training=self.training)
    x = x + inputs  # skip connection
return x

In [234]:
# MBConvBlock

blocks = nn.ModuleList([])
for block_args in blocks_args:
    # Update block input and output filters based on depth multiplier.
    block_args = block_args._replace(
        input_filters=round_filters(
            block_args.input_filters, global_params
        ),
        # input_filters=8,
        output_filters=round_filters(
            block_args.output_filters, global_params
        ),
        num_repeat=round_repeats(block_args.num_repeat, global_params),
    )

    # The first block needs to take care of stride and filter size increase.
    blocks.append(
        MBConvBlock(block_args, global_params, image_size=image_size)
    )
    print('Before: ', image_size)
    image_size = calculate_output_image_size_(image_size, block_args.stride)
    print('After: ', image_size)

Before:  [132, 15]
After:  [66, 8]
Before:  [66, 8]
After:  [33, 8]
Before:  [33, 8]
After:  [17, 4]


In [118]:
image_size

[17, 4]

In [119]:
# Head
in_channels = block_args.output_filters  # output of final block
out_channels = round_filters(128, global_params)
_conv_head = Conv2d(in_channels, out_channels, kernel_size=(2, 1), bias=False)
_bn1 = nn.BatchNorm2d(
    num_features=out_channels, momentum=bn_mom, eps=bn_eps
)

In [120]:
_dropout = nn.Dropout(global_params.dropout_rate)
_fc = nn.Linear(out_channels, global_params.num_classes)

# set activation to memory efficient swish by default
_swish = MemoryEfficientSwish()

In [121]:
# Stem
x0 = torch.randn((1, 1, 263, 15))
x1 = _conv_stem(x0)
x2 = _bn0(x1)
x3 = _swish(x2)
x3.shape

torch.Size([1, 8, 132, 15])

In [262]:
x = x3

In [263]:
print(len(blocks))

for idx, block in enumerate(blocks):
    print(x.shape)
    drop_connect_rate = global_params.drop_connect_rate
    if drop_connect_rate:
        drop_connect_rate *= float(idx) / len(blocks)
    x = block(x, drop_connect_rate=drop_connect_rate)
    print(x.shape)

3
torch.Size([1, 8, 132, 15])
torch.Size([1, 16, 66, 8])
torch.Size([1, 16, 66, 8])
torch.Size([1, 32, 33, 8])
torch.Size([1, 32, 33, 8])
torch.Size([1, 64, 17, 4])


In [265]:
x.shape

torch.Size([1, 64, 17, 4])

In [266]:
block_args

BlockArgs(num_repeat=1, kernel_size=(6, 3), stride=(2, 2), expand_ratio=2, input_filters=32, output_filters=64, se_ratio=0.25, id_skip=True)

In [271]:
in_channels = block_args.output_filters  # output of final block
out_channels = round_filters(128, global_params)
# out_channels = round_filters(1280, self._global_params) <-- 원본
# out_channels = 1280
_conv_head = Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
_bn1 = nn.BatchNorm2d(
    num_features=out_channels, momentum=bn_mom, eps=bn_eps)

In [273]:
x = _conv_head(x)
x = _bn1(x)
x = _swish(x)
x.shape

torch.Size([1, 128, 17, 4])

In [ ]:
# x = _avg_pooling(x)

In [295]:
out_channels

128

In [292]:
_avg_pooling = nn.AdaptiveAvgPool2d(1)
_dropout = nn.Dropout(global_params.dropout_rate)
_fc = nn.Linear(out_channels, global_params.num_classes)

# set activation to memory efficient swish by default
_swish = MemoryEfficientSwish()

In [293]:
x_copy = x

In [294]:
y = _avg_pooling(x)
y = y.flatten(start_dim=1)
y = _dropout(y)
y = _fc(y)
y

tensor([[-0.3333]], grad_fn=<AddmmBackward0>)

In [283]:

y = y.view(1, -1)
y.shape
# print(x.shape)

# x = x.flatten(start_dim=1)
# x = self._dropout(x)

# x = self._fc(x)

torch.Size([1, 128])

In [284]:
y

tensor([[0.1952, 0.2066, 0.1998, 0.2097, 0.2002, 0.2060, 0.2034, 0.2010, 0.1997,
         0.2035, 0.2030, 0.1976, 0.1983, 0.1964, 0.2027, 0.2046, 0.2035, 0.2029,
         0.2082, 0.2109, 0.2003, 0.2054, 0.2026, 0.2103, 0.1970, 0.2057, 0.1945,
         0.2000, 0.1880, 0.2057, 0.1968, 0.2006, 0.2054, 0.2093, 0.1914, 0.2065,
         0.2077, 0.2035, 0.2096, 0.1974, 0.2065, 0.1984, 0.2047, 0.2102, 0.2056,
         0.2111, 0.2112, 0.2067, 0.2039, 0.2021, 0.2035, 0.2015, 0.1834, 0.1905,
         0.2059, 0.2052, 0.2052, 0.1892, 0.2001, 0.1991, 0.2050, 0.2024, 0.2093,
         0.2104, 0.2007, 0.2100, 0.1988, 0.1895, 0.1999, 0.2009, 0.2069, 0.2135,
         0.1948, 0.2116, 0.1986, 0.1980, 0.1937, 0.1970, 0.2007, 0.2006, 0.1952,
         0.2038, 0.2047, 0.2028, 0.1999, 0.1964, 0.2048, 0.2022, 0.2015, 0.2061,
         0.2032, 0.2024, 0.1970, 0.1846, 0.1995, 0.1991, 0.2061, 0.2078, 0.1917,
         0.2001, 0.2058, 0.1946, 0.2025, 0.1998, 0.2066, 0.2030, 0.1995, 0.2035,
         0.1996, 0.2079, 0.1